In [ ]:
!pip install pyppeteer
import asyncio
from pyppeteer import launch

async def main():
    browser = await launch(headless=True)
    page = await browser.newPage()
    await page.goto('https://www.wantgoo.com/stock/etf/0050/constituent')
    await page.waitForSelector('table')  # 等待表格加載完成
    content = await page.evaluate('document.querySelector("table").innerText')
    print(content)
    await browser.close()

asyncio.get_event_loop().run_until_complete(main())

In [ ]:
!pip install selenium webdriver-manager pandas

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

try:
    # 設置 ChromeDriver 服務
    service = Service(ChromeDriverManager().install())
    
    # 設置瀏覽器選項
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-notifications')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    # 添加 user-agent
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    
    # 添加實驗性選項
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    # 啟動瀏覽器
    driver = webdriver.Chrome(service=service, options=options)
    
    # 設定瀏覽器腳本
    driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
        'source': '''
            Object.defineProperty(navigator, 'webdriver', {
                get: () => undefined
            })
        '''
    })
    
    # 打開目標網址 https://www.wantgoo.com/index/%5E539/stocks https://www.wantgoo.com/index/%5E543/stocks
    url = "https://www.wantgoo.com/index/%5E539/stocks"
    driver.get(url)
    
    # 等待頁面加載
    time.sleep(10)
    
    try:
        # 嘗試定位表格 (使用多個可能的選擇器)
        table = None
        selectors = [
            "//table[contains(@class, 'constituent-list')]",
            "//div[contains(@class, 'table')]//table",
            "//table"
        ]
        
        for selector in selectors:
            try:
                table = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, selector))
                )
                if table:
                    break
            except:
                continue
        
        if table:
            # 取得表格內容
            rows = table.find_elements(By.TAG_NAME, "tr")
            
            # 提取表頭
            headers = [th.text for th in rows[0].find_elements(By.TAG_NAME, "th")]
            print("Headers:", headers)
            
            # 提取數據
            data = []
            for row in rows[1:]:
                cols = row.find_elements(By.TAG_NAME, "td")
                if cols:
                    row_data = [col.text for col in cols]
                    data.append(row_data)
                    print("Row data:", row_data)
            
            # 創建 DataFrame
            df = pd.DataFrame(data, columns=headers)
            print("\nParsed DataFrame:\n")
            print(df)
            
            # 保存為 CSV
            df.to_csv("etf_0050_data.csv", index=False, encoding="utf-8-sig")
            print("\nData saved to 'etf_0050_data.csv'.")
        else:
            print("無法找到表格元素")
        
        # 等待用戶確認
        input("按Enter鍵結束...")
        
    except Exception as e:
        print(f"處理表格時發生錯誤: {e}")
        print("目前頁面內容:", driver.page_source[:500])  # 只印出前500字元
        
except Exception as e:
    print(f"設置時發生錯誤: {e}")
    
finally:
    try:
        driver.quit()
    except:
        pass

Headers: ['股票', '股價', '漲跌幅', '周漲幅', '開盤', '最高', '最低', '昨收', '成交量', '新高/低', '周轉率', '自選']
Row data: ['光寶科\n2301', '104', '▲2.5 (2.46%)', '2.97%', '102', '106', '102', '101.5', '17,720', '1日', '0.75%', '']
Row data: ['華南金\n2880', '27.1', '▲0.3 (1.12%)', '3.04%', '26.7', '27.4', '26.7', '26.8', '24,849', '創新高', '0.18%', '']
Row data: ['台灣大\n3045', '112', '▲1 (0.9%)', '0%', '111.5', '112.5', '111', '111', '2,753', '創新高', '0.07%', '']
Row data: ['世芯-KY\n3661', '3100', '▲25 (0.81%)', '3.51%', '3075', '3145', '3050', '3075', '1,910', '5日', '2.37%', '']
Row data: ['遠傳\n4904', '87.2', '▲0.5 (0.58%)', '-1.8%', '87.2', '88.2', '87.1', '86.7', '2,793', '-1日', '0.08%', '']
Row data: ['上海商銀\n5876', '40.8', '▲0.1 (0.25%)', '1.75%', '40.45', '41.2', '40.4', '40.7', '7,613', '-15日', '0.16%', '']
Row data: ['玉山金\n2884', '27.35', '▲0.05 (0.18%)', '1.48%', '27.3', '27.85', '27.25', '27.3', '22,877', '創新高', '0.14%', '']
Row data: ['富邦金\n2881', '91.1', '0 (0%)', '0%', '91.3', '92.3', '90.9', '91.1', '12,578'